In [1]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
from transformers import AutoImageProcessor, AutoModel
import torch
import os
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity 
from tqdm import tqdm
import pandas as pd
from PIL import Image

Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu126 for torchao version 0.13.0


In [2]:
# map
with open("../data/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("../data/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])
    
# loading the relevance judgements
with h5py.File("../data/london_lite_gt.h5","r") as f:
   fovs = f["fov"][:]
   sim = f["sim"][:].astype(np.uint8)

In [3]:
from huggingface_hub import login
login(new_session=False)

In [4]:
def recall(ranks, pidx, ks):
    recall_at_k = np.zeros(len(ks))
    for qidx in range(ranks.shape[0]):
        for i, k in enumerate(ks):
            if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:
                recall_at_k[i:] += 1
                break

    recall_at_k /= ranks.shape[0]
    return recall_at_k

def apk(pidx, rank, k):
    if len(rank)>k:
        rank = rank[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(rank):
        if p in pidx and p not in rank[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(pidx), k)

def mapk(ranks, pidxs, k):

    return np.mean([apk(a,p,k) for a,p in zip(pidxs, ranks)])

def mapk_many(ranks, pidxs, ks):
    return np.array([mapk(ranks, pidxs, k) for k in ks], dtype=float)

def average_precision(relevant, retrieved):
   precisions = []
   rel = 0
   for i in range(0, len(retrieved)):
      if retrieved[i] in relevant:
         rel += 1
         precisions.append(rel/(i+1))
   return sum(precisions) / len(relevant)

def mean_average_precision(all_relevant, all_retrieved):
   total = 0
   count = 0
   for qid in all_relevant: 
      total += average_precision(all_relevant[qid], all_retrieved.get(qid, []))
      count += 1
   return total / count


def l2_normalize(x, axis=1, eps=1e-12):
   norm = np.linalg.norm(x, axis=axis, keepdims=True)
   return x / (norm + eps)

def get_relevant_images(gt_similarity_matrix, query_idx):
   return np.where(gt_similarity_matrix[query_idx, :] == 1)[0]

def get_retrieved_images(feature_matrix, query_idx):
   return np.argsort(-feature_matrix[query_idx])

def save_results_to_csv(model_name, map_value, recall_at_k, mAPs, csv_path="./results/feature_extraction_evaluation.csv"):
    results_dict = {
        "models_name": model_name,
        "MAP": map_value * 100,
        "Recall@1": recall_at_k[0] * 100,
        "Recall@5": recall_at_k[1] * 100,
        "Recall@10": recall_at_k[2] * 100,
        "Recall@20": recall_at_k[3] * 100,
        "mAP@1": mAPs[0] * 100,
        "mAP@5": mAPs[1] * 100,
        "mAP@10": mAPs[2] * 100,
        "mAP@20": mAPs[3] * 100
    }

    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        if model_name in df['models_name'].values:
            df.loc[df['models_name'] == model_name] = pd.Series(results_dict)
        else:
            df = pd.concat([df, pd.DataFrame([results_dict])], ignore_index=True)
    else:
        df = pd.DataFrame([results_dict])
    
    df.to_csv(csv_path, index=False)
    return df


In [5]:
DINOV3_MODELS = {
    # ViT models
    "facebook/dinov3-vits16-pretrain-lvd1689m": 384,
    "facebook/dinov3-vits16plus-pretrain-lvd1689m": 384,
    "facebook/dinov3-vitb16-pretrain-lvd1689m": 768,
    "facebook/dinov3-vitl16-pretrain-lvd1689m": 1024,
    "facebook/dinov3-vith16plus-pretrain-lvd1689m": 1280,
    #"facebook/dinov3-vit7b16-pretrain-lvd1689m": 4096,  
    
    # ConvNeXt models
    "facebook/dinov3-convnext-tiny-pretrain-lvd1689m": 768,
    "facebook/dinov3-convnext-small-pretrain-lvd1689m": 768,
    "facebook/dinov3-convnext-base-pretrain-lvd1689m": 1024,
    "facebook/dinov3-convnext-large-pretrain-lvd1689m": 1536,
    
    # SAT pretrained models
    "facebook/dinov3-vitl16-pretrain-sat493m": 1024,
    #"facebook/dinov3-vit7b16-pretrain-sat493m": 4096, 
}

In [7]:
for MODEL_TO_USE, feat_dim in DINOV3_MODELS.items():
    print(f"\n{'='*80}")
    print(f"Loading model: {MODEL_TO_USE}")
    print(f"{'='*80}\n")
    
    try:
        processor = AutoImageProcessor.from_pretrained(MODEL_TO_USE)
        model = AutoModel.from_pretrained(MODEL_TO_USE) 
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
    except Exception as e:
        print(e)
        continue
    if os.path.exists(f"./results/pooling_comparison/{MODEL_TO_USE.replace('/', '_')}_pooling_comparison.csv"):
        continue
    m_feats_cls = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    m_feats_mean = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    m_feats_mean_no_cls = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    m_feats_max = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    m_feats_gem = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    
    p = 3.0
    
    for i, img_name in enumerate(tqdm(m_imgs)):
        img = Image.open(os.path.join('../data02/', img_name))
        inputs = processor(images=img, return_tensors="pt").to(device)
        
        with torch.inference_mode():
            outputs = model(**inputs)
        
        m_feats_cls[i] = outputs.pooler_output[0].cpu().numpy()
        m_feats_mean[i] = outputs.last_hidden_state.mean(dim=1)[0].cpu().numpy()
        m_feats_mean_no_cls[i] = outputs.last_hidden_state[:, 1:, :].mean(dim=1)[0].cpu().numpy()
        m_feats_max[i] = outputs.last_hidden_state.max(dim=1)[0][0].cpu().numpy()
        m_feats_gem[i] = outputs.last_hidden_state[:, 1:, :].clamp(min=1e-6).pow(p).mean(dim=1).pow(1./p)[0].cpu().numpy()
    
    q_feats_cls = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
    q_feats_mean = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
    q_feats_mean_no_cls = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
    q_feats_max = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
    q_feats_gem = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
    
    print("Extracting query features...")
    for i, img_name in enumerate(tqdm(q_imgs)):
        img = Image.open(os.path.join('../data02/', img_name)) 
        inputs = processor(images=img, return_tensors="pt").to(device)
        
        with torch.inference_mode():
            outputs = model(**inputs)
        
        q_feats_cls[i] = outputs.pooler_output[0].cpu().numpy()
        q_feats_mean[i] = outputs.last_hidden_state.mean(dim=1)[0].cpu().numpy()
        q_feats_mean_no_cls[i] = outputs.last_hidden_state[:, 1:, :].mean(dim=1)[0].cpu().numpy()
        q_feats_max[i] = outputs.last_hidden_state.max(dim=1)[0][0].cpu().numpy()
        q_feats_gem[i] = outputs.last_hidden_state[:, 1:, :].clamp(min=1e-6).pow(p).mean(dim=1).pow(1./p)[0].cpu().numpy()
    
    
    m_feats_cls = l2_normalize(m_feats_cls, axis=1)
    m_feats_mean = l2_normalize(m_feats_mean, axis=1)
    m_feats_mean_no_cls = l2_normalize(m_feats_mean_no_cls, axis=1)
    m_feats_max = l2_normalize(m_feats_max, axis=1)
    m_feats_gem = l2_normalize(m_feats_gem, axis=1)
    
    q_feats_cls = l2_normalize(q_feats_cls, axis=1)
    q_feats_mean = l2_normalize(q_feats_mean, axis=1)
    q_feats_mean_no_cls = l2_normalize(q_feats_mean_no_cls, axis=1)
    q_feats_max = l2_normalize(q_feats_max, axis=1)
    q_feats_gem = l2_normalize(q_feats_gem, axis=1)
    
    pooling_strategies = {
        "CLS_token": (q_feats_cls, m_feats_cls),
        "Mean_pooling": (q_feats_mean, m_feats_mean),
        "Mean_no_CLS": (q_feats_mean_no_cls, m_feats_mean_no_cls),
        "Max_pooling": (q_feats_max, m_feats_max),
        "GeM_pooling": (q_feats_gem, m_feats_gem),
    }
    
    all_results = []
    best = None
    best_map = 0.0
    best_recall_at_k = None
    best_mAPs = None
    
    for pooling_name, (q_feats, m_feats) in pooling_strategies.items():
        print(f"{'='*60}")
        print(f"Evaluating: {pooling_name}")
        print(f"{'='*60}")
        
        similarities = cosine_similarity(q_feats, m_feats)
        
        ranks = np.argsort(-similarities, axis=1)  # (Q, G)
        
        Q = similarities.shape[0]
        all_rel = {}
        pidx = []
        for q in range(Q):
            all_rel[q] = get_relevant_images(sim, q)
            pidx.append(np.array(all_rel[q], dtype=int))
        
        all_ret = {q: ranks[q] for q in range(Q)}
        
        ks = [1, 5, 10, 20]
        recall_at_k = recall(ranks, pidx, ks)
        mAPs = mapk_many(ranks, pidx, ks)
        map_value = mean_average_precision(all_rel, all_ret)
        
        # Track best
        if best is None or map_value > best_map:
            best_map = map_value
            best = pooling_name
            best_recall_at_k = recall_at_k
            best_mAPs = mAPs
        
        print(f"MODEL: {MODEL_TO_USE}")
        print(f"Pooling: {pooling_name}")
        print(f"MAP: {map_value*100:.2f}%")
        for k, r, m in zip(ks, recall_at_k, mAPs):
            print(f"Recall@{k}: {r*100:.2f}%   mAP@{k}: {m*100:.2f}%")
        print("\n")
        
        result_dict = {
            "models_name": f"{MODEL_TO_USE}_{pooling_name}",
            "pooling_strategy": pooling_name,
            "MAP": map_value * 100,
            "Recall@1": recall_at_k[0] * 100,
            "Recall@5": recall_at_k[1] * 100,
            "Recall@10": recall_at_k[2] * 100,
            "Recall@20": recall_at_k[3] * 100,
            "mAP@1": mAPs[0] * 100,
            "mAP@5": mAPs[1] * 100,
            "mAP@10": mAPs[2] * 100,
            "mAP@20": mAPs[3] * 100
        }
        all_results.append(result_dict)
    
    df_results = pd.DataFrame(all_results)
    CSV_PATH = f"./results/pooling_comparison/{MODEL_TO_USE.replace('/', '_')}_pooling_comparison.csv"
    os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
    df_results.to_csv(CSV_PATH, index=False)

    df_main = save_results_to_csv(f"{MODEL_TO_USE}_{best}", best_map, best_recall_at_k, best_mAPs)
    
    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("\n")



Loading model: facebook/dinov3-vits16-pretrain-lvd1689m


Loading model: facebook/dinov3-vits16plus-pretrain-lvd1689m


Loading model: facebook/dinov3-vitb16-pretrain-lvd1689m


Loading model: facebook/dinov3-vitl16-pretrain-lvd1689m


Loading model: facebook/dinov3-vith16plus-pretrain-lvd1689m


Loading model: facebook/dinov3-convnext-tiny-pretrain-lvd1689m


Loading model: facebook/dinov3-convnext-small-pretrain-lvd1689m


Loading model: facebook/dinov3-convnext-base-pretrain-lvd1689m


Loading model: facebook/dinov3-convnext-large-pretrain-lvd1689m


Loading model: facebook/dinov3-vitl16-pretrain-sat493m



In [8]:
df_final = pd.read_csv("./results/feature_extraction_evaluation.csv")
print("\n=== FINAL MODEL COMPARISON (Best pooling for each) ===")
print(df_final[['models_name', 'MAP', 'Recall@1', 'Recall@20']].sort_values('MAP', ascending=False).to_string(index=False))


=== FINAL MODEL COMPARISON (Best pooling for each) ===
                                                 models_name       MAP  Recall@1  Recall@20
    facebook/dinov3-vith16plus-pretrain-lvd1689m_GeM_pooling 36.461502      44.0       82.4
        facebook/dinov3-vitb16-pretrain-lvd1689m_GeM_pooling 34.103124      41.4       85.4
        facebook/dinov3-vitl16-pretrain-lvd1689m_Max_pooling 34.050443      40.4       80.2
          facebook/dinov3-vits16-pretrain-lvd1689m_CLS_token 30.767844      38.0       77.6
      facebook/dinov3-vits16plus-pretrain-lvd1689m_CLS_token 29.901573      33.6       73.6
                   openai/clip-vit-large-patch14_Mean_no_CLS 28.641709      32.6       80.4
                    openai/clip-vit-base-patch32_GeM_pooling 28.097766      38.8       82.2
                            geolocal/StreetCLIP_Mean_pooling 25.725327      30.0       73.8
                   openai/clip-vit-base-patch16_Mean_pooling 24.915458      31.0       76.8
 facebook/dinov3-convnex